<a href="https://colab.research.google.com/github/tejabalu/udub/blob/master/Technin510_L4_PartI_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 4: Image Classification

## Step 1: Create project/folder and download data

Download [lab-4.zip](https://drive.google.com/file/d/1vjt-_R1YnpIkxoytykU_ZZ5CLNP7fVgi/view?usp=sharing) and unzip it. Place the `train/` and `test/` folders into `lab-4/` folder you create on your Google drive. These folders include images of five 'symbol' cards as seen from a small robot's camera. Also copy `lab4.ipynb` into the `lab-4/`.

## Step 2: Implement feature extraction

Below is a skeleton code for an image classification class called ImageClassifier as well as code for creating, training, and testing a classifier with the provided data sets. The three functions you will need to implement are indicated with comments in the code.

The first one of these is `extract_image_features` which should return a Numpy array that contains the features that represent the image. Before extracting any features, you should apply Gaussian blurring to your images to get rid of random sensor noice that is common in many lower cost cameras. For this, look into the filters module of scikit-image. Then explore at least two different types of features provided in the feature module of scikit-image. Inspect the size of the feature vectors generated by different methods and what the features look like for different images from the training or testing set. 

You will not yet get a good sense of how well each feature performs in allowing the classifiers to discriminate between different images. Hence, keep your code for extracting different features around until you explore classification performance in the next step. You can do that by duplicating the function with different names for different features.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import re

from sklearn import svm, metrics
from skimage import io, feature, filters, exposure, color, exposure
from skimage.color import rgb2gray
from skimage.feature import hog
from skimage.feature import canny

import cv2


from sklearn.externals import joblib

from google.colab import drive
drive.mount('/content/drive')

class FeatureExtractor:
    
    def __init__(self):
        self.classifier = None
        self.folder = '/content/drive/My Drive/lab4-data/lab4-data/'

    def imread_convert(self, f):
        return io.imread(f).astype(np.uint8)

    def save_classifier(self):
        joblib.dump(self.classifier, self.folder + 'classifier.joblib')

    def load_data_from_folder(self, dir):
        # read all images into an image collection
        ic = io.ImageCollection(self.folder + dir + '*.bmp',
                                load_func=self.imread_convert)

        # create one large array of image data
        data = io.concatenate_images(ic)
        
        # extract labels from image names
        labels = np.array(ic.files)
        for i, f in enumerate(labels):
            m = re.search('_', f)
            labels[i] = (f[len(dir):m.start()]).split('/')[-1]
        
        return(data,labels)

    def extract_image_features(self, data, feature):
        
        
      
        # apply greayscale 
        grayscale = rgb2gray(data)

        # apply gaussian
        filtered_images = filters.gaussian(grayscale)

        # print(filtered_images.shape)

        featured_images = []



        for img in data:
          featured_image = feature(img[:,:,0]).flatten()
          featured_images.append(featured_image)

        featured_images = np.array(featured_images)

        return featured_images

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Here is the code for creating an instance of the class, loading the data, and extracting the features.

In [12]:
img_clf = FeatureExtractor()

# load images
print('Loading training set...')
(train_raw, train_labels) = img_clf.load_data_from_folder('train/')
print('Loading testing set...')
(test_raw, test_labels) = img_clf.load_data_from_folder('test/')
print()



Loading training set...
Loading testing set...



In [14]:
# convert images into features, example using hog features
print('Extracting HOG features...')
train_data_hog = img_clf.extract_image_features(train_raw, hog)
test_data_hog = img_clf.extract_image_features(test_raw, hog)

# repeat with at least one other feature type...
print('Extracting CANNY features...')
train_data_canny = img_clf.extract_image_features(train_raw, canny)
test_data_canny = img_clf.extract_image_features(test_raw, canny)

Extracting HOG features...
Extracting CANNY features...


In [15]:
# Inspecting the features.
print('HOG Feature Vector')
print(train_data_hog)
print(f'Vector Size: {len(train_data_hog[0])}')
print(train_data_hog.shape)

# repeat with at least one other feature type...
print('Canny Feature Vector')
print(train_data_canny)
print(f'Vector Size: {len(train_data_canny[0])}')
print(train_data_canny.shape)

HOG Feature Vector
[[0.05667177 0.01060595 0.0262972  ... 0.00799891 0.08782366 0.05636069]
 [0.08722828 0.06733667 0.0257614  ... 0.08341104 0.04451854 0.03370315]
 [0.         0.         0.         ... 0.01569028 0.         0.02631339]
 ...
 [0.         0.         0.         ... 0.09145972 0.05971634 0.02465507]
 [0.08530864 0.05746551 0.05076483 ... 0.06805885 0.04292549 0.00815909]
 [0.08271605 0.01973666 0.01273122 ... 0.         0.         0.0243155 ]]
Vector Size: 86184
(196, 86184)
Canny Feature Vector
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
Vector Size: 76800
(196, 76800)


## Step 3: Implement classifier training and testing
Next, implement the `train_classifier` and `predict_labels` functions for at least two different types of classifiers. After this, the second cell below shoul then produce performance results of the classifier.

Explore the performance of at least **two feature types** and at least **two classifiers** (i.e. at least four different combinations) in terms of classification **performance** on the test set. Update the code to display the **F1 score** on the test set for the **four different combinations** with informative prompts. Then the code should display the **detailed performance** (confusion matrix, accuracy, F1 score) that is already displayed only for the best performing combination of features and classifier. Also make sure the best performing classifier is saved onto your drive for the next step of the lab.

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import f1_score 

# Continue implementation of the ImageClassifier class in this cell
class ImageClassifier(FeatureExtractor): # we are linking both the classes imgvclassifier has all the methods from the featureextractor 

    def train_classifier(self, train_data, train_labels, classifier):
        
        self.classifier = classifier()
        self.classifier.fit(train_data, train_labels)

#here train data will be the features from the above 
    def predict_labels(self, data):
        
        predicted_labels = self.classifier.predict(data)
        
        return predicted_labels        


Re-run initialization and feature extraction, then train/test the classifier.

In [20]:
img_clf = ImageClassifier()

# create list of list containing features you want to run through along with any
# associated metadata

# features = 
features = [hog, canny]
# testfeatures = [test_data_hog , test_data_canny]

# create list of list of classifiers you want to use 

# classifiers = 
classifiers = [KNeighborsClassifier, MLPClassifier, DecisionTreeClassifier, GaussianNB]

# Variable to hold the best F1 score
best_f1 = 0  

# Loop through the different combinantions of features and classifiers
for i in features:

  # extract features for train raw using current feature value
  itrain = img_clf.extract_image_features(train_raw, i)
  itest = img_clf.extract_image_features(test_raw, i)
  # extract features for test raw using curent feature value
  # (What we know: train_labels and test_labels from above)
  for classifier in classifiers:  
    print(i)
    print(classifier)
    

    # Train model
    model = img_clf.train_classifier(itrain, train_labels, classifier)

    # Test model
    predictetdlabels = img_clf.predict_labels(itest)
    
    # Create confusion matrix
    confmatrx = confusion_matrix(test_labels, predictetdlabels)

    # Print test accuracy
    accscore = accuracy_score(test_labels, predictetdlabels)
    print("Accuracy:"+str(accscore))

    # Print test F1 score
    f1 = f1_score(test_labels, predictetdlabels, average='macro')
    print("F1 Score:"+str(f1))
    
    # Check if last model is better than the current best one and save it
    if f1>best_f1:
      best_f1=f1
      img_clf.save_classifier()
      print("Best classifier updated..")
      print(classifier)
      
    print()
    
    # pass # delete this line

<function hog at 0x7f77cc2be560>
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Accuracy:0.8
F1 Score:0.7997474747474748
Best classifier updated..
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>

<function hog at 0x7f77cc2be560>
<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
Accuracy:0.825
F1 Score:0.8269751082251082
Best classifier updated..
<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>

<function hog at 0x7f77cc2be560>
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
Accuracy:0.525
F1 Score:0.5233585858585859

<function hog at 0x7f77cc2be560>
<class 'sklearn.naive_bayes.GaussianNB'>
Accuracy:0.65
F1 Score:0.6909722222222222

<function canny at 0x7f77d0336b00>
<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
Accuracy:0.125
F1 Score:0.027777777777777776

<function canny at 0x7f77d0336b00>
<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
Accuracy:0.375
F1 Score:0.

In [ ]:
# TODO: Save the best model using the `save_classifier` method of the `FeatureExtractor` class.

## Step 4: Transfer classifier to your camera

Next you will apply your trained classifier directly onto images captured by your webcam. Since this is our last lab using Python we would like to give you the opportunity to install Python and Jupyter notebooks on your machine, and do this part of the lab as a conventional Python script. However, if you would rather not do that at the moment, you still have the option of doing this part with Colab Notebooks.

* **Option 1:** If you would like to do this part locally, first follow the Python Installation guidelines. Then download the skeleton camera capture script [`camera.py`](https://drive.google.com/file/d/1jmll8_rsagFLcaaIpKtYgCd2Rn8rVXlL/view?usp=sharing) and test it out. Then update this script as described below and submit this script. Make sure you note in lab4.ipynb that you chose this option.
* **Option 2:** If you would like to do this part in Colab Notebooks, add your code below. Check out [`camera.ipynb`](https://colab.research.google.com/drive/1IfHqK83dDVyxsQzQwsnxc4CUrRUoL9y8) for sample code for capturing camera images in Colab notebooks.

Your code should first load the classifier you saved in Step 3 with the following line (already implemented in the sample code):

`classifier = joblib.load('classifier.joblib')`

The script should then go into a loop where it (1) captures a new image from the camera, (2) processes the image to make it grayscale and filter the noise, (3) extracts the features like you did in Step 2 using the right set of features for your trained classifier, (4) detects whether the image contains one of the seven images using the trained classifier, and (5) displays the detected class name on the image in every iteration. You can use images printed on paper or displayed on your phone.

In [25]:
from IPython.display import HTML, Audio
from IPython.display import clear_output
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io as io2
from PIL import Image
import time
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

############################
####### CAMERA CODE ########
############################

VIDEO_HTML = """
<video autoplay
 width=%d height=%d style='cursor: pointer;'></video>
<script>

var video = document.querySelector('video')

navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

function getFrame() {
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    return canvas.toDataURL('image/jpeg', 0.8)
}

</script>
"""

# Make sure the pictures taken by your camera match the size of the 
# training set – 320px x 240px
def start_camera(filename='photo.jpg', quality=0.8, size=(320,240)):
  display(HTML(VIDEO_HTML % (size[0],size[1])))

def take_photo(filename='photo.jpg', quality=0.8, size=(320,240)):
  data = eval_js('getFrame()')
  binary = b64decode(data.split(',')[1])
  f = io2.BytesIO(binary)
  return np.asarray(Image.open(f))

############################
############################
############################

class ImageClassifer_webcam(ImageClassifier):
  
  def load_classifier(self):
    print('Loading classifier...')
    self.classifier = joblib.load(self.folder + 'classifier.joblib')
    print(self.classifier)

# Create a new instance of the classifier
img_clf = ImageClassifer_webcam()

# Load the previously saved model
img_clf.load_classifier()

# Start camera and wait for it to "warm up"
start_camera()
time.sleep(3)

while(True):
  img = take_photo() # click

  ##############################################################################
  ############################ YOUR CODE HERE ##################################
  ##############################################################################

  # Convert to grayscale
  grayscale = rgb2gray(img)
  print(grayscale.shape)


  # Extract features from the image just caputred
  imgf = hog(grayscale)

  # Show frame and prediction
  pred = img_clf.predict_labels(imgf.reshape(1, -1))
  print(pred)
  
  time.sleep(2)

  # break # delete this line
 

Loading classifier...
MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)


(240, 320)
['none']
(240, 320)
['hands']
(240, 320)
['hands']
(240, 320)
['hands']
(240, 320)
['none']
(240, 320)
['hands']
(240, 320)
['none']
(240, 320)
['drone']
(240, 320)
['hands']
(240, 320)
['hands']
(240, 320)
['hands']
(240, 320)
['hands']
(240, 320)
['hands']
(240, 320)
['place']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320)
['none']
(240, 320

KeyboardInterrupt: ignored

## (Optional) Improve classifier

You will notice that your classifier is prone to errors when tested with your camera. Part of the reason is that the images were collected from a different camera. You can try improving the performance of your camera image classifier by re-training your classifier with images collected from your camera.

## Step 5: Submit your code on Canvas

Complete this lab by submitting a link to your updated Colab Notebook (and if you chose Option 1 in Step 4, by uploading your updated `camera.py`) on Canvas, by Oct 22 Tuesday, 11:59pm. We will test your code by running it and inspecting the classification results to make sure: 
* A comparison of  at least four combinations of feature types and classifiers were made
* A reasonable classification performance was achieved with the best combination (higher than random chance).

We will test your camera image classification code by running it and showing it the seven different printed images to check that more than half of the images can be recognized correctly in some configuration relative to the camera.

See Canvas for a grading rubric.